# Controle da carteira de ações (Swing Trade)

- **Este Notebook é só para gerar o dataset usado para fazer o IR e acampanhar o rendimento da carteira**

- **Não funciona para Day Trade.**


## Baixando a planinlha com os ativos negociados

Para pegar a planinlha de excel com todas as negociações é só ir no site da [B3](http://www.b3.com.br/pt_br/), procurar pelo Canal Eletrônico do Investidor (CEI).

> [Link direto](https://cei.b3.com.br/CEI_Responsivo/login.aspx) 

Depois de acessar a página é só ir em: Extratos e informativos $\Rightarrow$ Negociações de Ativos. Você deve ver a opção de baixar a planilha por excel nesta página.

# Inserir os arquivos da B3 (InfoCEI.xls) e csv

In [1]:
# Lista com o nome dos arquivos InfoCEI ou o .csv de uma carteira antiga.

arq_names = ['InfoCEI.xls', 'InfoCEI (1).xls']#, 'Ultima_carteira_2019.csv']
des_name = 'desdobramento_agrupamento_2019.xls'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Funções

In [3]:
def cleaning_and_organizing(df):

    df.rename(columns = {'Preço (R$)':'Preço de C/V', 'Data Negócio': 'Data',
                         'Valor Total (R$)':'Valor Total',
                         'Especificação do Ativo':'Especificação',
                         'C/V':'Tipo de Execução',
                         'Quantidade': 'Quantidade de C/V'}, inplace=True);

    df['Mercado'] = df['Mercado'].apply(lambda x: x.replace('Merc.',''))
    df['Quantidade de C/V'] = df['Quantidade de C/V'].astype(np.int16)
    df['Código'] = df['Código'].apply(lambda x: x[:-1])
    df['Tipo de Execução'] = df['Tipo de Execução'].str.strip()
    df['Mercado'] = df['Mercado'].str.strip()
    df['Especificação'] = df['Especificação'].str.strip()
    df['Código'] = df['Código'].str.strip()
    df['Corretora'] = df_ativos['Corretora'].apply(lambda x: x.split('-')[1].strip().split(' ')[0])
    
    df.loc[df['Tipo de Execução'] == 'V', ['Quantidade de C/V']] *= -1
    
    return df

In [4]:
def calc_price_and_gain(tipo_op, preco_atual_anterior, preco_cv, qnt_atual_anterior, qnt_cv):

    if tipo_op == 'C':

        mean = np.average([preco_atual_anterior, preco_cv], weights = [qnt_atual_anterior, qnt_cv]) 
        return mean, 0.0

    elif tipo_op == 'V':

        mean = preco_atual_anterior
        lucro = (preco_cv - preco_atual_anterior) * np.abs(qnt_cv)
        return mean, lucro

    elif (tipo_op == 'DES') | (tipo_op == 'AGR'):

        mean = preco_atual_anterior * preco_cv
        return mean, 0.0
    
    return np.nan

In [5]:
def stock_mean_price(df_acao):
    
    ''' Calcula o preço médio da ação e em caso de venda o lucro/prejuízo. '''
    
    preco_cv = df_acao['Preço de C/V']

    quantidade_cv = df_acao['Quantidade de C/V']

    quantidade_atual = df_acao['Quantidade Atual']

    tipo_op = df_acao['Tipo de Execução']

    mean_price = np.zeros(df_acao.shape[0], dtype=np.float64)
    gain_loss = np.zeros(df_acao.shape[0], dtype=np.float64)

    mean_price[0] = preco_cv[0]

    for i in range(1, df_acao.shape[0]):

        mean_price[i], gain_loss[i] = calc_price_and_gain(tipo_op[i],
                                                          mean_price[i - 1], preco_cv[i],
                                                          quantidade_atual[i - 1], quantidade_cv[i])
    
    return mean_price[:], gain_loss[:]        

In [6]:
def combine_InfoCEI_files(arq_names):
    
    df = pd.DataFrame()
    
    for name in arq_names:
            
        if name.find('.csv') != -1:
            
            aux = pd.read_csv(name)
        
            aux['Data Negócio'] = pd.to_datetime(aux['Data Negócio'])
        
            df = pd.concat([df, aux], ignore_index=True)            
            
        elif name.find('.xls') != -1:
                
            aux = pd.read_excel(name, header=10, skipfooter=4).dropna(axis=1)
            
            aux['Data Negócio'] = aux['Data Negócio'].apply(lambda x: x.replace('/','-'))
            aux['Data Negócio'] = pd.to_datetime(aux['Data Negócio'], infer_datetime_format=True, dayfirst=True)
            
            aux['Corretora'] = pd.read_excel(name, header=8, nrows=1).dropna(axis=1).iloc[0, 0]
            
            df = pd.concat([df, aux], ignore_index=True)
    
    df.sort_values(by='Data Negócio', ascending=True, inplace=True)
    
    return df.reset_index(drop=True)

In [7]:
def save_last(columns, df):
    
    df_carteira = pd.DataFrame(columns = columns)
    
    carteira = carteira_de_acoes_atual(df, offline=True)
    
    df_carteira['Data Negócio'] = carteira['Data']
    df_carteira['C/V'] = 'C'
    df_carteira['Quantidade'] = carteira['Quantidade Atual']
    df_carteira['Código'] = carteira['Código'].apply(lambda x: str(x) + 'F')
    df_carteira['Preço (R$)'] = carteira['<Preço>']
    df_carteira['Valor Total (R$)'] = carteira['<Valor Total>']
    df_carteira['Mercado'] = 'Merc. Fracionário'
    df_carteira['Corretora'] = carteira['Corretora']
    
    
    for cod in df_carteira['Código']:

        df_carteira.loc[df_carteira['Código'] == cod, 'Especificação do Ativo'] = df_all.loc[df_all['Código'] == cod, 'Especificação do Ativo'].iloc[-1]

    return df_carteira

In [8]:
def carteira_de_acoes_atual(df, offline=False):
    
    carteira = pd.DataFrame(columns=df.columns)

    cods = df['Código'].unique()
    
    for cod in cods:

        last_idx = df[df[['Código']] == cod].last_valid_index() 

        carteira = carteira.append(df.iloc[last_idx], ignore_index=True)

    carteira = carteira[carteira['Quantidade Atual'] != 0.0]
    
    carteira.sort_values(by='Data', ascending=True, inplace=True)
    
    drop_columns = ['Especificação', 'Tipo de Execução', 'Quantidade de C/V', 'Mercado', 'Preço de C/V']
    
    carteira.drop(drop_columns, inplace=True, axis=1)
    
    carteira.rename(columns={'Preço Atual': '<Preço>'}, inplace=True)
    
    carteira.reset_index(drop=True, inplace=True)
    
    carteira['<Valor Total>'] = carteira['Quantidade Atual'] * carteira['<Preço>']
    
    if offline == False:
    
        carteira['Preço Atual'] = get_market_values(carteira['Código'])

        carteira['Valor Total Atual'] = carteira['Quantidade Atual'] * carteira['Preço Atual']

        carteira['L/P'] = carteira['Quantidade Atual'] * ( carteira['Preço Atual'] - carteira['<Preço>'] )

        carteira['L/P (%)'] = (carteira['L/P'] / (carteira['<Preço>'] * carteira['Quantidade Atual'])) * 100.0

        carteira['L/P'] = carteira['L/P'].round(2)

        carteira['L/P (%)'] = carteira['L/P (%)'].round(2) 

        carteira['<Preço>'] = carteira['<Preço>'].round(2)


        cols_ordenadas = ['Data', 'Código', 'Quantidade Atual', '<Preço>', '<Valor Total>',
                          'Preço Atual', 'Valor Total Atual', 'L/P', 'L/P (%)', 'Corretora']

    elif offline == True:
        
        cols_ordenadas = ['Data', 'Código', 'Quantidade Atual', '<Preço>', '<Valor Total>', 'Corretora']
        
    return carteira[cols_ordenadas]

In [9]:
def intersection(l1, l2):
    return list(set(l1) & set(l2))

# Lê os arquivos InfoCEI e csv

In [10]:
df_all = combine_InfoCEI_files(arq_names)

df_ativos = df_all.copy()
df_des_agr = pd.read_excel(des_name)

WARNING *** file size (30195) not 512 + multiple of sector size (512)
WARNING *** file size (30195) not 512 + multiple of sector size (512)
WARNING *** file size (32477) not 512 + multiple of sector size (512)
WARNING *** file size (32477) not 512 + multiple of sector size (512)


In [11]:
df_ativos.head()

,Data Negócio,C/V,Mercado,Código,Especificação do Ativo,Quantidade,Preço (R$),Valor Total (R$),Corretora
0,2019-07-11,C,Merc. Fracionário,RAIL3F,RUMO S.A. ON NM,20,22.17,443.40,1982 - MODAL DTVM LTDA
1,2019-07-11,C,Merc. Fracionário,PETR4F,PETROBRAS PN N2,17,28.17,478.89,1982 - MODAL DTVM LTDA
2,2019-07-11,C,Merc. Fracionário,CVCB3F,CVC BRASIL ON NM,10,53.00,530.00,1982 - MODAL DTVM LTDA
3,2019-07-11,C,Merc. Fracionário,VALE3F,VALE ON NM,10,52.00,520.00,1982 - MODAL DTVM LTDA
4,2019-07-19,V,Merc. Fracionário,CVCB3F,CVC BRASIL ON NM,8,48.70,389.60,1982 - MODAL DTVM LTDA


# Organiza e limpa o dataset

In [12]:
# Limpando e arruamndo os dados
df_ativos = cleaning_and_organizing(df_ativos)
df_des_agr['Data'] = pd.to_datetime(df_des_agr['Data'], infer_datetime_format=True, dayfirst=True)

In [13]:
# Tira a média diária das vendas ou compras. Isso é necessário porque podemos comprar
# ações com valores diferentes durante o pregão em um mesmo dia.

group = df_ativos.groupby(['Data', 'Código', 'Mercado', 'Especificação', 'Tipo de Execução', 'Corretora'])

df_ativos = group[['Quantidade de C/V', 'Valor Total']].sum().reset_index()

df_ativos['Preço de C/V'] = np.abs(df_ativos['Valor Total'] / df_ativos['Quantidade de C/V'].astype(np.float64))

# Adicionando algumas colunas.
df_ativos['Preço Atual'] = 0.0
df_ativos['L/P'] = 0.0
df_ativos['Quantidade Atual'] = 0.0

# Faz as contas

In [14]:
cods = intersection(df_ativos['Código'].unique(), df_des_agr['Código'].unique())

for cod in cods:

    df = df_des_agr[df_des_agr['Código'] == cod]
    
    for _, (data_split, _, prop1, prop2) in df.iterrows():

        mask = (df_ativos['Código'] == cod) & (df_ativos['Data'] < data_split)

        row = df_ativos[mask].tail(1).copy()
        
        # Taxas para converter os valores que serão desdobrados/agrupados.
        ratio1 = prop1 / float(prop2)
        ratio2 = prop2 / float(prop1)

        if ratio1 > ratio2:
            row['Tipo de Execução'] = 'DES'
        else:
            row['Tipo de Execução'] = 'AGR'
        
        row['Quantidade de C/V'] = df_ativos.loc[mask, 'Quantidade de C/V'].sum() * (ratio1 - 1.0)
        row['Valor Total'] = ratio1
        row['Preço de C/V'] = ratio2
        
        row.reset_index(inplace=True)
        row['index'] += 1
        row.set_index('index', inplace=True)

        mask = (df_ativos['Data'] < data_split)
        
        df_ativos.reset_index(inplace=True)
        df_ativos.loc[-mask, 'index'] += 1
        df_ativos.set_index('index', inplace=True)

        df_ativos = df_ativos.append(row).sort_values(by='index')
        df_ativos.reset_index(drop=True, inplace=True)
        del row

In [15]:
cods = df_ativos['Código'].unique()

for cod in cods:
    
    mask = (df_ativos['Código'] == cod)

    df_ativos.loc[mask, 'Quantidade Atual'] = df_ativos.loc[mask, 'Quantidade de C/V'].cumsum()

    mean_price, gain_loss = stock_mean_price(df_ativos[mask].reset_index(drop=True))

    df_ativos.loc[mask, 'Preço Atual'] = mean_price
    df_ativos.loc[mask, 'L/P'] = gain_loss

# Salva o dataset final com as movimentações diárias

In [16]:
#df_ativos.to_csv('Dataset_ações_2019.csv', index=False)

# Salva a última carteira

In [17]:
#df_carteira = save_last(df_all.columns, df_ativos.copy())
#df_carteira.to_csv('Ultima_carteira_2019.csv', index=False)